## 인공지능 HW2
### -문장을 TF-IDF 높은 단어를 기준으로 임베딩 후 k-means-
#### 담당 교수님: 김학수 교수님
#### 201711719 응용통계학과 심은선
#### 제출일: 2020.09.11
####               

문장 집합으로부터 단어(공백 기준, 심볼 제거)을 추출하고, 각 단어의 TF*IDF 가중치 계산

가중치 상위 200개를 선택하여 vocabulary를 생성

Vocabulary를 참조하여 각 문장을 200차원의 벡터로 표현
  예) Vocabulary = [apple, cluster, filter, spam, zoo]
       Input: This is a spam filter. ==> [0, 0, 1, 1, 0]

K-Means(k=2)를 수행한 후, 각 문장 별로 범주명(1 또는 2) 출력


In [1]:
import re
import numpy as np
import collections
from typing import List
import sys
np.set_printoptions(threshold=sys.maxsize) #모두 출력하기 위해 설정

from sklearn.cluster import KMeans

In [2]:
# 파일 읽기
def read_files(file_path):
    texts = []
    with open(file_path, 'r', encoding='utf-8') as inFile:
        lines = inFile.readlines()
        for line in lines:
            texts.append(line)
            
    return texts


# 정규표현식으로 숫자, 특수문자 제외, 소문자로 통일
def preprocess(docs: List):
    preprocessed_texts = []
    for text in docs:
        preprocessed_texts.append(re.sub('[^a-z^A-Z]', ' ', text).lower())
    
#     print(preprocessed_texts)
    return preprocessed_texts

# 문서별 TF 구하기 (문서별로 단어 등장 횟수 count, 공백 기준 분리)
def get_TF(docs: List):
    TF = [] 
    for idx, text in enumerate(docs):
        words = text.split()
        word_count = collections.Counter(words)
        TF.append(word_count)
    
    return TF


# DF 구하기
def get_DF(docs: List):
    word_ls = []
    for doc in docs:
        word_ls += list(set(doc.split()))
    df = collections.Counter(word_ls) #문서 전체의 df
    return df



# 문서별 TF-IDF 구하기
def get_TFIDF(file_path):
    docs = read_files(file_path)
    preprocessed_docs = preprocess(docs)
    tf = get_TF(preprocessed_docs)
    df = get_DF(preprocessed_docs)
    N = len(docs)
    
    docs_tf_idf = []
    for word_count in tf: #문서마다 반복
        doc_tf_idf = dict()
        for word, count in word_count.items(): # 문서의 단어별로 tf-idf를 구한다
            word_tf_idf = count * np.log(N / df[word])
            doc_tf_idf[word] = word_tf_idf
        docs_tf_idf.append(doc_tf_idf)
    
    return docs_tf_idf



# 문서들의 tf_df 합치기
def integrate_tf_idf(docs_tf_idf: List):
    total_tf_idf = collections.defaultdict(int)
    for tf_idf in docs_tf_idf:
        for word in tf_idf:
            total_tf_idf[word] += tf_idf[word] #동일 단어는 tf_idf 합 사용
    
    return total_tf_idf



# tf_idf가 높은 상위 k개의 단어 리스트 얻기
def get_freq_word(tf_idf: dict, k: int):
    sort_docs_tf_idf = list(sorted(tf_idf.items(), key=lambda x: x[1], reverse=True))[:k]
    vocab = list(zip(*sort_docs_tf_idf))[0]
    vocab = list(vocab)
    vocab.sort()
    
    return vocab



# 단어 리스트 기반 voca-idx 딕셔너리 생성
def word_to_idx(words: List):
    voca_dict = {}
    for idx, voca in enumerate(words):
        voca_dict[voca] = idx
    
    return voca_dict



# 문장을 voca 포함 여부(0/1)로 임베딩하기 (tf-idf 상위 k개의 vocabulary 기반 임베딩(0/1))
def embed_docs(file_path: str, k: int):
    raw_docs = read_files(file_path)
    docs = preprocess(raw_docs)
    docs_tf_idf = get_TFIDF(file_path)
    total_tf_idf = integrate_tf_idf(docs_tf_idf)
    voca_dict = word_to_idx(get_freq_word(total_tf_idf, k))
    N = len(raw_docs) #문서 개수(=문장 개수)
#     print(voca_dict)
    
    docs_embed = np.zeros((N, k)) #문장 임베딩(벡터)의 모임 -> 행렬
    for i, doc in enumerate(docs):
        words = doc.split()
        for word in words:
            if word in voca_dict: #단어를 포함할 때만 해당 부분을 1로 바꾼다
                idx = voca_dict[word]
                docs_embed[i][idx] = 1
    
    return docs_embed

In [3]:
file_path = './SMS_dataset.txt'
embedded_docs = embed_docs(file_path, 200)

In [5]:
#K-means Clustering
clf_km = KMeans(n_clusters=2, random_state=0) #하이퍼파라미터 = cluster 개수
y_km = clf_km.fit_predict(embedded_docs) #학습시키고 각각의 cluster 리턴

In [6]:
#Clustering 결과 출력하기 - 문장별 범주 출력
print(y_km+1)

[1 1 2 1 2 2 2 2 2 2 2 2 2 2 1 2 1 1 1 2 1 2 1 1 1 2 1 1 1 1 2 1 1 2 2 2 1
 1 1 2 2 2 2 1 1 1 1 1 1 2 1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 2 2 2 1 2 1 1 1 1
 1 1 1 1 1 1 1 1 2 1 1 1 1 2 2 1 2 2 2 1 2 2 1 2 2 2 1 2 2 2 1 1 1 1 2 1 1
 1 1 2 1 2 2 1 1 2 2 1 2 2 1 1 2 1 1 1 1 1 1 2 2 1 1 1 2 2 1 1 1 1 1 1 2 1
 1 1 1 1 1 2 2 1 1 2 2 2 1 1 1 2 1 1 2 2 1 1 1 2 1 1 1 1 2 2 1 2 2 1 1 1 1
 1 1 1 1 1 1 2 1 2 1 1 1 1 2 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 2 2 1 2
 1 1 2 2 1 1 1 1 2 2 1 1 2 1 1 1 1 1 1 1 2 1 1 1 2 1 2 2 1 1 1 2 1 2 2 1 2
 2 1 1 1 2 1 1 2 2 2 2 2 2 2 1 1 1 2 1 1 1 1 1 1 1 1 1 1 2 2 1 2 1 1 1 1 2
 1 2 1 1 1 1 2 1 1 1 1 1 2 1 1 2 1 1 1 1 1 1 2 1 2 1 1 2 1 1 1 1 1 2 1 1 1
 2 2 1 1 1 1 1 1 2 1 2 1 1 1 1 2 1 1 1 1 2 1 2 2 2 1 1 1 1 1 2 2 1 1 2 1 1
 1 1 1 2 1 1 1 1 1 1 2 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 2 2 2 2 2 1 1 1 2 1
 1 1 1 1 2 1 1 1 2 1 2 2 2 1 2 1 1 2 1 1 1 1 2 1 2 1 1 1 1 1 1 1 2 2 2 1 1
 1 1 1 1 1 1 1 1 2 2 1 1 1 1 1 1 2 2 1 1 2 1 2 2 1 1 2 1 2 2 1 2 1 1 1 1 1
 1 1 2 2 1 2 2 1 1 1 1 1 